## Things covered in this module:



1.Novelty Detection 

2.Validity Resoning 



In [ ]:
!pip install accelerate protobuf sentencepiece torch git+https://github.com/huggingface/transformers huggingface_hub
!pip install llama-index-llms-huggingface
!pip install -U langchain-community
!pip install llama-index-embeddings-langchain
!pip install sentence-transformers

!pip install llama-index

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os 
os.environ['TOKENIZERS_PARALLELISM'] = "True"

In [6]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

In [7]:
documents=SimpleDirectoryReader("/kaggle/input/demo-db").load_data()
documents

[Document(id_='04bb4af2-fc53-445a-8cdc-160500f53414', embedding=None, metadata={'page_label': '1', 'file_name': 'a survey of ai t2i and t2v.pdf', 'file_path': '/kaggle/input/demo-db/a survey of ai t2i and t2v.pdf', 'file_type': 'application/pdf', 'file_size': 403891, 'creation_date': '2024-08-13', 'last_modified_date': '2024-08-13'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='A Survey of AI Text-to-Image and AI Text-to-Video Generators Aditi Singh  Department of Computer Science Kent State University Ohio, USA asingh37@kent.edu Abstract— Text-to-Image and Text-to-Video AI generation models are revolutionary technologies that use deep learning and natural language processing (NLP) techniques to create images and videos from textual descriptions. This

In [8]:
system_prompt="""
You are a advanced research assistant. Your goal is to understand the paper given and provide the summary of it 
mentioning its novelty along with papers referred in it  as
accurately as possible based on the instructions and context provided.
"""
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [10]:
!nvidia-smi

Tue Aug 13 09:18:39 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
!pip install --upgrade transformers


In [12]:
import transformers 

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map=0,
    model_kwargs={"torch_dtype": torch.float16 }
)
  

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [13]:
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
2024-08-13 09:19:52.761306: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 09:19:52.761424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 09:19:52.890494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS fact

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [15]:
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c3bf9cef640>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c3bf9cef640>, id_func=<function default_id_func at 0x7c3f967c0310>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.core.service_context_elements.llama_logger.LlamaLogger object at 0x7c3f831c6f20>, callback_manager=<llama_index

In [16]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)
     

In [17]:
index

In [18]:
query_engine=index.as_query_engine(do_sample=True)

In [37]:
summary =query_engine.query("what is the summary of the paper t2speech?")

In [38]:
summary

Response(response='\nThe paper "T2Speech" presents a high-level diagram of a text-to-speech (TTS) system, which takes text as input and generates audio output. The system consists of several components, including a preprocessor, encoder, decoder, and vocoder. The preprocessor tokenizes the input text into words, breaks them into phonemes based on their pronunciation, and represents the phoneme duration and pitch. The encoder inputs the linguistic features (phonemes) and outputs an n-dimensional embedding. The decoder converts the embedded information to an acoustic feature (mel-spectrogram), which is then converted to audio output by the vocoder. The authors propose a two-stage approach, where the decoder first predicts the mel-spectrogram from the latent processed features and then generates audio using the mel-spectrogram. The paper also discusses the importance of prosody in TTS and how it can be achieved through the use of energy, pitch, and duration predictors.\n\nAs for the novel

In [39]:
summary.response

'\nThe paper "T2Speech" presents a high-level diagram of a text-to-speech (TTS) system, which takes text as input and generates audio output. The system consists of several components, including a preprocessor, encoder, decoder, and vocoder. The preprocessor tokenizes the input text into words, breaks them into phonemes based on their pronunciation, and represents the phoneme duration and pitch. The encoder inputs the linguistic features (phonemes) and outputs an n-dimensional embedding. The decoder converts the embedded information to an acoustic feature (mel-spectrogram), which is then converted to audio output by the vocoder. The authors propose a two-stage approach, where the decoder first predicts the mel-spectrogram from the latent processed features and then generates audio using the mel-spectrogram. The paper also discusses the importance of prosody in TTS and how it can be achieved through the use of energy, pitch, and duration predictors.\n\nAs for the novelty of the paper, i

In [40]:
novel = query_engine.query("what is the so unique about this paper  t2speech? tell me in bullet points ")

In [41]:
novel

Response(response='\n\nThe paper "T2Speech: Text-to-Speech with Neural Voices" (page 1) presents several novel aspects in the field of text-to-speech (TTS) systems:\n\n1. **Neural Voices:** The paper introduces the concept of "Neural Voices," which are generated using deep neural networks. These voices sound more natural than standard voices and produce human-like speech patterns, such as stress and loudness of individual words.\n2. **Custom Neural Voice:** The paper proposes a custom neural voice option that uses the user\'s own audio data to create a one-of-a-kind, customized synthetic voice. This level of voice personalization is not commonly found in TTS systems.\n3. **Deepest Level of Voice Personalization:** Custom Neural Voice offers the deepest level of voice personalization, with realistic speech that can be used to represent brands, personify machines, and allow users to interact with applications conversationally.\n4. **Phoneme, Prosody, and Mel-spectrogram:** The paper expl

In [42]:
novel.response

'\n\nThe paper "T2Speech: Text-to-Speech with Neural Voices" (page 1) presents several novel aspects in the field of text-to-speech (TTS) systems:\n\n1. **Neural Voices:** The paper introduces the concept of "Neural Voices," which are generated using deep neural networks. These voices sound more natural than standard voices and produce human-like speech patterns, such as stress and loudness of individual words.\n2. **Custom Neural Voice:** The paper proposes a custom neural voice option that uses the user\'s own audio data to create a one-of-a-kind, customized synthetic voice. This level of voice personalization is not commonly found in TTS systems.\n3. **Deepest Level of Voice Personalization:** Custom Neural Voice offers the deepest level of voice personalization, with realistic speech that can be used to represent brands, personify machines, and allow users to interact with applications conversationally.\n4. **Phoneme, Prosody, and Mel-spectrogram:** The paper explains the importanc

In [43]:
validity = query_engine.query("Does this paper have any value in current trends or scenarios ? if so list out and also explain why with matching trends or tech")

In [44]:
validity

Response(response='\nBased on the provided context, the paper "Text to Speech Conversion" published in Indian Journal of Science and Technology in October 2016, has some value in current trends and scenarios. Here are some reasons why:\n\n1. Deep Learning-based OCR: The paper discusses the use of deep learning-based Optical Character Recognition (OCR) techniques for text to speech conversion. This is a current trend in the field of Natural Language Processing (NLP) as deep learning-based models have shown better performance in OCR tasks compared to traditional machine learning-based approaches.\n2. Conversational AI: The paper proposes a text-to-speech conversion system that can convert text to speech in real-time, which is a key component of conversational AI. With the growing interest in conversational AI, the paper\'s approach could be useful in developing chatbots and virtual assistants that can communicate with users in a more natural way.\n3. Voice Assistants: The paper\'s focus 

In [45]:
validity.response

'\nBased on the provided context, the paper "Text to Speech Conversion" published in Indian Journal of Science and Technology in October 2016, has some value in current trends and scenarios. Here are some reasons why:\n\n1. Deep Learning-based OCR: The paper discusses the use of deep learning-based Optical Character Recognition (OCR) techniques for text to speech conversion. This is a current trend in the field of Natural Language Processing (NLP) as deep learning-based models have shown better performance in OCR tasks compared to traditional machine learning-based approaches.\n2. Conversational AI: The paper proposes a text-to-speech conversion system that can convert text to speech in real-time, which is a key component of conversational AI. With the growing interest in conversational AI, the paper\'s approach could be useful in developing chatbots and virtual assistants that can communicate with users in a more natural way.\n3. Voice Assistants: The paper\'s focus on text-to-speech 